**Import required libraries and scripts**

In [1]:
import os
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.performance_calculation import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *


[11:20:53] Initializing Normalizer
/home/alacournola/anaconda3/envs/dockm8/lib/python3.8/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")
[TRJ.py:171 - <module>()] netCDF4 is not available. Writing AMBER ncdf files will be slow.
/home/alacournola/anaconda3/envs/dockm8/lib/python3.8/site-packages/MDAnalysis/coordinates/TRJ.py:1209: DeprecationWarning: Please use `netcdf_file` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated.
  class NCDFPicklable(scipy.io.netcdf.netcdf_file):


In [2]:
for dir in os.listdir('/home/alacournola/DEKOIS-noprot/'):
    print(dir)
    software = '/home/alacournola/DockM8/software'
    protein_file = f'/home/alacournola/DEKOIS-noprot/{dir}/receptor_protoss_prepared.pdb'
    ref_file = f'/home/alacournola/DEKOIS-noprot/{dir}/crystal_ligand_protoss.sdf'
    pocket = 'reference'
    protonation = 'none'
    docking_library = f'/home/alacournola/DEKOIS-noprot/{dir}/merged_actives_decoys.sdf'
    docking_programs = ['GNINA', 'SMINA', 'PLANTS']
    clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
    clustering_method = 'KMedoids'
    rescoring= ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'SCORCH', 'RTMScore', 'vinardo']
    id_column = 'ID'
    n_poses = 10
    exhaustiveness = 8
    #ncpus = 1
    ncpus = int(os.cpu_count()-2)
    #Create a temporary folder for all further calculations
    w_dir = os.path.dirname(protein_file)
    print('The working directory has been set to:', w_dir)
    create_temp_folder(w_dir+'/temp')
    
    try:
        calculate_EF_single_functions(w_dir, docking_library, clustering_metrics)
        apply_consensus_methods_combinations(w_dir, docking_library, clustering_metrics)
    except Exception as e:
        printlog(f'Failed for {dir}')
        print(e)
        


aldr
The working directory has been set to: /home/alacournola/DEKOIS-noprot/aldr
The folder: /home/alacournola/DEKOIS-noprot/aldr/temp already exists
The folder: /home/alacournola/DEKOIS-noprot/aldr/temp/ranking already exists
The folder: /home/alacournola/DEKOIS-noprot/aldr/temp/consensus already exists
The folder: /home/alacournola/DEKOIS-noprot/aldr/temp/ranking already exists
The folder: /home/alacournola/DEKOIS-noprot/aldr/temp/consensus already exists

[2023-May-12 11:20:58]: Calculating consensus methods for every possible score combination...


100%|██████████| 8/8 [04:13<00:00, 31.74s/it]


hdac2
The working directory has been set to: /home/alacournola/DEKOIS-noprot/hdac2
The folder: /home/alacournola/DEKOIS-noprot/hdac2/temp already exists
The folder: /home/alacournola/DEKOIS-noprot/hdac2/temp/ranking already exists
The folder: /home/alacournola/DEKOIS-noprot/hdac2/temp/consensus was created
The folder: /home/alacournola/DEKOIS-noprot/hdac2/temp/ranking already exists
The folder: /home/alacournola/DEKOIS-noprot/hdac2/temp/consensus already exists

[2023-May-12 11:25:20]: Calculating consensus methods for every possible score combination...


100%|██████████| 8/8 [04:14<00:00, 31.75s/it]


cdk2
The working directory has been set to: /home/alacournola/DEKOIS-noprot/cdk2
The folder: /home/alacournola/DEKOIS-noprot/cdk2/temp already exists
The folder: /home/alacournola/DEKOIS-noprot/cdk2/temp/ranking already exists
The folder: /home/alacournola/DEKOIS-noprot/cdk2/temp/consensus was created
The folder: /home/alacournola/DEKOIS-noprot/cdk2/temp/ranking already exists
The folder: /home/alacournola/DEKOIS-noprot/cdk2/temp/consensus already exists

[2023-May-12 11:29:42]: Calculating consensus methods for every possible score combination...


100%|██████████| 8/8 [04:14<00:00, 31.80s/it]


igf1r
The working directory has been set to: /home/alacournola/DEKOIS-noprot/igf1r
The folder: /home/alacournola/DEKOIS-noprot/igf1r/temp already exists
The folder: /home/alacournola/DEKOIS-noprot/igf1r/temp/ranking already exists
The folder: /home/alacournola/DEKOIS-noprot/igf1r/temp/consensus was created
The folder: /home/alacournola/DEKOIS-noprot/igf1r/temp/ranking already exists
The folder: /home/alacournola/DEKOIS-noprot/igf1r/temp/consensus already exists

[2023-May-12 11:34:03]: Calculating consensus methods for every possible score combination...


100%|██████████| 8/8 [04:19<00:00, 32.42s/it]


nram
The working directory has been set to: /home/alacournola/DEKOIS-noprot/nram
The folder: /home/alacournola/DEKOIS-noprot/nram/temp already exists
The folder: /home/alacournola/DEKOIS-noprot/nram/temp/ranking already exists
The folder: /home/alacournola/DEKOIS-noprot/nram/temp/consensus was created
The folder: /home/alacournola/DEKOIS-noprot/nram/temp/ranking already exists
The folder: /home/alacournola/DEKOIS-noprot/nram/temp/consensus already exists

[2023-May-12 11:38:31]: Calculating consensus methods for every possible score combination...


100%|██████████| 8/8 [04:12<00:00, 31.51s/it]


In [ ]:
def rename_columns_in_csv(root_dir, target_file, col_rename_dict):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if target_file in filename:
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath)
                df.rename(columns=col_rename_dict, inplace=True)
                df.to_csv(filepath, index=False)
                print(f"Columns renamed in: {filepath}")

root_directory = "/media/mario/T7/FINISHED/FINISHED/"
target_csv_file = "allposes_rescored.csv"
column_rename_dict = {
    "AD4_Affinity": "AD4",
    "Vinardo_Affinity": "Vinardo",
    "GNINA_Affinity": "GNINA",
    "GNINA_CNN_Score": "CNN-Score",
    "GNINA_CNN_Affinity": "CNN-Affinity",
    'LinF9_Affinity':'LinF9',
    'SCORCH_pose_score':'SCORCH',
    # Add more column names to be renamed here
}
rename_columns_in_csv(root_directory, target_csv_file, column_rename_dict)

In [ ]:
import os
import pandas as pd

def rename_and_merge_csv_files(root_dir, target_file, output_file):
    merged_df = None

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == target_file:
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath, index_col=0)
                dirname = os.path.basename(dirpath)
                print(dirpath.replace('/media/mario/T7/FINISHED/FINISHED/', '').replace('/temp/consensus', ''))
                df.rename(columns={"EF1%": dirpath.replace('/media/mario/T7/FINISHED/FINISHED/', '').replace('/temp/consensus', '')}, inplace=True)
                df.drop(columns='EF10%', inplace=True)
                if merged_df is None:
                    merged_df = df
                else:
                    merged_df = pd.merge(merged_df, df, on=["Scoring Function", "Clustering Metric"])

    if merged_df is not None:
        # Adding the average column
        columns_to_exclude = ['method_name', 'selected_columns', 'clustering_metric']
        numeric_columns = [col for col in merged_df.columns if col not in columns_to_exclude]
        merged_df['Average'] = merged_df[numeric_columns].mean(axis=1)
        merged_df.to_csv(output_file)
        print(f"Merged CSV file saved to: {output_file}")


root_directory = "/media/mario/T7/FINISHED/FINISHED/"
target_csv_file = "EF_single_functions.csv"
output_csv_file = "merged_output_DUD-E.csv"

rename_and_merge_csv_files(root_directory, target_csv_file, output_csv_file)


In [ ]:
import os
import pandas as pd

def rename_and_merge_csv_files(root_dir, target_file, output_file):
    merged_df = None

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == target_file:
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath, index_col=0)
                dirname = os.path.basename(dirpath)
                df.rename(columns={"EF1%": dirpath.replace('/media/mario/T7/FINISHED/FINISHED/', '').replace('/temp/consensus', '')}, inplace=True)
                if merged_df is None:
                    merged_df = df
                else:
                    merged_df = pd.merge(merged_df, df, on=["clustering_method", "selected_columns", 'method_name'])
                    print(merged_df.head())

    if merged_df is not None:
        # Adding the average column
        columns_to_exclude = ['method_name', 'selected_columns', 'clustering_metric']
        numeric_columns = [col for col in merged_df.columns if col not in columns_to_exclude]
        merged_df['Average'] = merged_df[numeric_columns].mean(axis=1)
        print(merged_df.head())
        merged_df.to_csv(output_file)
        print(f"Merged CSV file saved to: {output_file}")

root_directory = "/media/mario/T7/FINISHED/FINISHED/"
target_csv_file = "consensus_summary.csv"
output_csv_file = "merged_output_consensus_DUD-E.csv"

rename_and_merge_csv_files(root_directory, target_csv_file, output_csv_file)
